In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

device = 'cuda' if torch.cuda.is_available() else 'cpu'
batch_size = 64
epochs = 3

train_set = datasets.MNIST(root='./data', train=True, download=True, transform=ToTensor())
test_set = datasets.MNIST(root='./data', train=False, download=True, transform=ToTensor())

train_loader = DataLoader(train_set, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_set, shuffle=False, batch_size=batch_size)

class CNNMnist(nn.Module):
    def __init__(self):
        super(CNNMnist, self).__init__()
        self.net = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d((2, 2), stride=2),
            nn.Conv2d(64, 128, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d((2, 2), stride=2),
            nn.Conv2d(128, 64, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d((2, 2), stride=2)
        )
        self.classification_head = nn.Sequential(
            nn.Linear(64, 20, bias=True),
            nn.Sigmoid(),
            nn.Linear(20, 10, bias=True)
        )

    def forward(self, x):
        features = self.net(x)
        x = self.classification_head(features.view(x.size(0), -1))
        return x

class ModifiedCNNMnist(nn.Module):
    def __init__(self):
        super(ModifiedCNNMnist, self).__init__()
        self.net = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d((2, 2), stride=2),
            nn.Conv2d(32, 64, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d((2, 2), stride=2),
            nn.Conv2d(64, 32, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d((2, 2), stride=2)
        )
        self.classification_head = nn.Sequential(
            nn.Linear(32, 10, bias=True)  # Adjusted the number of output features to 10
        )

    def forward(self, x):
        features = self.net(x)
        x = self.classification_head(features.view(features.size(0), -1))
        return x

def train_one_epoch(model, train_loader, optimizer, loss_fn):
    model.train()
    total_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

def test_model(model, test_loader):
    model.eval()
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    accuracy = correct / len(test_loader.dataset)
    return accuracy

original_model = CNNMnist().to(device)
modified_model = ModifiedCNNMnist().to(device)

optimizer_original = torch.optim.SGD(original_model.parameters(), lr=0.001)
optimizer_modified = torch.optim.SGD(modified_model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

accuracy_original = []
accuracy_modified = []
percentage_drop = []

original_params = sum(p.numel() for p in original_model.parameters())

for epoch in range(epochs):
    train_loss = train_one_epoch(original_model, train_loader, optimizer_original, loss_fn)
    accuracy = test_model(original_model, test_loader)
    accuracy_original.append(accuracy)
    print(f"Epoch {epoch+1}: Original Model Train Loss: {train_loss:.4f}, Test Accuracy: {accuracy:.4f}")

    train_loss = train_one_epoch(modified_model, train_loader, optimizer_modified, loss_fn)
    accuracy = test_model(modified_model, test_loader)
    accuracy_modified.append(accuracy)
    print(f"Epoch {epoch+1}: Modified Model Train Loss: {train_loss:.4f}, Test Accuracy: {accuracy:.4f}")

    if epoch > 0:
        percentage_drop.append((original_params - sum(p.numel() for p in modified_model.parameters())) / original_params * 100)

plt.plot(percentage_drop, accuracy_original, label="Original Model")
plt.plot(percentage_drop, accuracy_modified, label="Modified Model")
plt.xlabel("Percentage Drop in Parameters")
plt.ylabel("Accuracy")
plt.title("Percentage Drop in Parameters vs Accuracy")
plt.legend()
plt.show()


Epoch 1: Original Model Train Loss: 2.3365, Test Accuracy: 0.0974
Epoch 1: Modified Model Train Loss: 2.2979, Test Accuracy: 0.1432
